In [200]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
from impyute.imputation.cs import fast_knn
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import re
from copy import deepcopy

In [201]:
columns_to_drop = [
    'Unnamed: 0',
    'Планировка',
    'Строительная серия',
    'Сдан',
    'Газоснабжение',
    'Аварийность',
    'Лифты',
    'Срок сдачи',
    'Отделка',
    'Подъезды',
    'id',
    'Тип перекрытий',
    'Мусоропровод'
]
df = pd.read_excel(r"C:\Users\ilyak\Downloads\data.xlsx").drop(columns_to_drop, axis=1)
indexes_of_rows_to_drop = []
for i, row in df.iterrows():
    total_na = row.isnull().sum()
    if total_na > 8:
        indexes_of_rows_to_drop.append(i)
df = df.drop(index=indexes_of_rows_to_drop, axis=0)
df = df.reset_index(drop=True)
df.shape

(1196, 17)

In [202]:
def count_bathrooms(s):
    result = 0
    for symbol in s:
        if symbol.isdigit():
            result += int(symbol)
    return result

In [203]:
"""
Куча страшно выглядящей предобработки
"""
df['Площадь'] = df['Общая'].apply(lambda s: float(s[:-3].replace(',', '.')) if type(s) == str else s)
df['price'] = df['price'].apply(lambda s: float(s[:-2].replace(',', '.').replace(' ', '')) if type(s) == str else s)
df['Новойстройка'] = df['Тип жилья'].apply(lambda s: s if type(s) != str else (1 if 'Новостройка' in s else 0))
df['Пентхаус'] = df['Тип жилья'].apply(lambda s: s if type(s) != str else (1 if 'Пентхаус' in s else 0))
df['Апартаменты'] = df['Тип жилья'].apply(lambda s: s if type(s) != str else (1 if 'Апартаменты' in s else 0))
df['Кухня'] = df['Кухня'].apply(lambda s: s if type(s) != str else float(s[:-3].replace(',', '.')))
df['Вид на улицу'] = df['Вид из окон'].apply(lambda s: s if type(s) != str else (1 if 'улицу' in s else 0))
df['Парковка'] = df['Парковка'].apply(lambda s: 'Наземная' if (type(s) != str or s == 'Открытая') else s)
df['Санузел'] = df['Санузел'].apply(lambda s: count_bathrooms(s) if type(s) == str else None)
df['Высота потолков'] = df['Высота потолков'].apply(lambda s: float(s[:-2].replace(',', '.')) if type(s) == str else s)
df['Ремонт'] = df['Ремонт'].apply(lambda s: s if type(s) == str else 'Без ремонта')
df['Жилплощадь'] = df['Жилая'].apply(lambda s: float(s[:-3].replace(',', '.')) if type(s) == str else s)
df['Балкон'] = df['Балкон/лоджия'].apply(lambda s: sum([(int(symb) if symb.isdigit() else 0) for symb in s]) if type(s) == str else 0)
df['Метро'] = df['sub'].apply(lambda s: s if type(s) != str else s.split()[0])
df['Время до метро'] = df['sub'].apply(lambda s: int(re.search(r'\d{1,2} мин.', s)[0][:-5]) if (type(s) == str and 'мин.' in s) else None)
df['Всего этажей'] = df['Этаж'].apply(lambda s: s if type(s) != str else int(s.split()[-1]))
df['Этаж'] = df['Этаж'].apply(lambda s: s if type(s) != str else int(s.split()[0]))
df = df.drop(['Тип жилья', 'Вид из окон', 'Балкон/лоджия', 'sub', 'Общая', 'Жилая'], axis=1)

In [204]:
"""
Здесь я перевожу качественные переменные в количественные.
Но т.к. я в дальнейшем хочу NaNы заполнить определенными знчениями, то я их не буду
кодировать числами, а просто оставлю NaNами.
Поэтому я применяю кодировку ко всему датафрейму, а затем возвращаю NaNы туда, где они были.
"""
labelencoder = LabelEncoder()
nan_labels_dict = {}
for column in ['Тип дома', 'Отопление', 'Парковка', 'Ремонт', 'Метро']: # это список столбцов с кач. переменными
    labelencoder.fit(df[column])
    quantitative = labelencoder.fit_transform(df[column])
    qualitative = df[column]
    df[column] = quantitative
    df_ = pd.DataFrame({'quan': quantitative, 'qual': qualitative})
    try:
        nan_label = df_[df_.qual.isnull()].quan.iloc[0]
    except IndexError:
        nan_label = None
    nan_labels_dict[column] = nan_label

for column in nan_labels_dict.keys():
    df[column] = df[column].apply(lambda n: None if n == nan_labels_dict[column] else n)
print(df.shape)
df.head()

(1196, 21)


,Кухня,price,Год постройки,Санузел,Тип дома,Отопление,Парковка,Высота потолков,Ремонт,Этаж,...,Площадь,Новойстройка,Пентхаус,Апартаменты,Вид на улицу,Жилплощадь,Балкон,Метро,Время до метро,Всего этажей
0,23.1,37821977.0,NaN,NaN,NaN,NaN,1,3.0,0,4,...,105.3,1,0,0,0.0,44.8,0,157.0,9.0,9
1,NaN,139000000.0,2016.0,3.0,3.0,NaN,1,4.5,1,4,...,200.0,0,0,1,1.0,NaN,0,16.0,6.0,6
2,21.0,29900000.0,2012.0,1.0,3.0,3.0,1,2.8,1,2,...,78.0,0,0,0,1.0,38.0,1,84.0,2.0,10
3,NaN,36000000.0,NaN,NaN,3.0,NaN,1,2.9,1,9,...,43.0,0,0,0,NaN,NaN,0,81.0,29.0,12
4,8.7,24800000.0,2021.0,2.0,4.0,1.0,1,2.7,2,3,...,75.7,0,0,0,1.0,51.4,0,133.0,19.0,14


In [205]:
"""
А теперь заменяем NaNы наиболее вероятными значениями.
Их мы находим с помощью метода ближайших соседей (то есть NaN будет заполнен тем же значением,
которое стоит у другого наблюдения из датасета с максимально похожими данными)
"""
columns = list(df.columns)
sys.setrecursionlimit(100000)
imputer = KNNImputer(n_neighbors=5)
imputer.fit(df.dropna())
df = pd.DataFrame(imputer.fit_transform(df))
df.columns = columns
for column in ['Кухня', 'price', 'Высота потолков', 'Площадь', 'Жилплощадь']:
    columns.remove(column)
for column in columns:
    df[column] = df[column].apply(round)
df

,Кухня,price,Год постройки,Санузел,Тип дома,Отопление,Парковка,Высота потолков,Ремонт,Этаж,...,Площадь,Новойстройка,Пентхаус,Апартаменты,Вид на улицу,Жилплощадь,Балкон,Метро,Время до метро,Всего этажей
0,23.10,37821977.0,1993,2,2,2,1,3.00,0,4,...,105.3,1,0,0,0,44.80,0,157,9,9
1,20.00,139000000.0,2016,3,3,3,1,4.50,1,4,...,200.0,0,0,1,1,141.80,0,16,6,6
2,21.00,29900000.0,2012,1,3,3,1,2.80,1,2,...,78.0,0,0,0,1,38.00,1,84,2,10
3,19.52,36000000.0,2007,2,3,3,1,2.90,1,9,...,43.0,0,0,0,1,62.40,0,81,29,12
4,8.70,24800000.0,2021,2,4,1,1,2.70,2,3,...,75.7,0,0,0,1,51.40,0,133,19,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,26.00,23367161.0,2017,2,3,2,1,3.05,0,19,...,114.9,1,0,1,0,67.40,1,163,18,24
1192,16.20,18300000.0,2021,1,3,1,2,2.95,0,11,...,50.1,0,0,0,1,31.30,1,168,5,20
1193,13.50,20914657.0,1984,2,2,2,1,2.95,0,4,...,67.2,1,0,0,1,41.00,0,181,6,32
1194,16.20,20378500.0,2011,1,3,3,1,3.00,0,14,...,76.9,1,0,1,0,36.66,2,87,11,21


In [206]:
df.to_excel(r"C:\Users\ilyak\Downloads\processed_data.xlsx", index=False)

In [207]:
# ТаДааааа